# Getting started with `tess-locator`

The `tess-locator` package is a fast and user-friendly tool to query TESS pixel coordinates and FFI filenames in a fast way without requiring internet access.  It builds upon the existing [`tess-point`](https://github.com/christopherburke/tess-point) and `tess-cloud` packages.

## Example 1: Converting celestial to pixel coordinates

The main feature of the package is the `locate()` function, which allows positions in the sky to be converted into TESS pixel coordinates.
You can enter the Simbad name of an object:

In [1]:
from tess_locator import locate
locate("Alpha Cen")

List of 3 coordinates
 ↳[TessCoord(sector=11, camera=2, ccd=2, column=1699.1, row=1860.3, time=None)
   TessCoord(sector=12, camera=2, ccd=1, column=359.9, row=1838.7, time=None)
   TessCoord(sector=38, camera=2, ccd=2, column=941.1, row=1953.7, time=None)]

You can pass an optional `time` or `sector` parameter if you are only interested in observations obtained at a specific time:

In [2]:
locate("Alpha Cen", time="2019-04-28")

List of 1 coordinates
 ↳[TessCoord(sector=11, camera=2, ccd=2, column=1699.1, row=1860.3, time=2019-04-28 00:00:00)]

In [3]:
locate("Alpha Cen", sector=12)

List of 1 coordinates
 ↳[TessCoord(sector=12, camera=2, ccd=1, column=359.9, row=1838.7, time=None)]

In addition to passing names, you can locate a custom `SkyCoord` object containing exact coordinates:

In [17]:
from astropy.coordinates import SkyCoord
locate(SkyCoord(ra=60, dec=70, unit='deg'), sector=19)

List of 1 coordinates
 ↳[TessCoord(sector=19, camera=2, ccd=2, column=355.3, row=1045.9, time=None)]

The *locate()* function returns a list of `TessCoord` objects which can be accessed using standard list and attribute syntax:

In [5]:
crd = locate("Alpha Cen")[0]
crd.sector, crd.camera, crd.ccd, crd.column, crd.row

(11, 2, 2, 1699.0540739785683, 1860.2510951146114)

You can also access the coordinates as a Pandas DataFrame:

In [6]:
locate("Alpha Cen").to_pandas()

,sector,camera,ccd,column,row
time,,,,,
NaN,11,2,2,1699.054074,1860.251095
NaN,12,2,1,359.913650,1838.696320
NaN,38,2,2,941.057771,1953.665513


## Example 2: Accessing FFI filenames for a pixel coordinate

When you have obtained a `TessCoord` object, you can use it to obtain a list of the TESS Full Frame Images (FFIs) which covered the position: 

In [7]:
crd.list_images()

List of 1248 images
 ↳[TessImage("tess2019113062933-s0011-2-2-0143-s_ffic.fits")
   TessImage("tess2019113065933-s0011-2-2-0143-s_ffic.fits")
   TessImage("tess2019113072933-s0011-2-2-0143-s_ffic.fits")
   TessImage("tess2019113075933-s0011-2-2-0143-s_ffic.fits")
   ...
   TessImage("tess2019140065932-s0011-2-2-0143-s_ffic.fits")
   TessImage("tess2019140072932-s0011-2-2-0143-s_ffic.fits")
   TessImage("tess2019140075932-s0011-2-2-0143-s_ffic.fits")
   TessImage("tess2019140082932-s0011-2-2-0143-s_ffic.fits")]

You can query the image list for a specific time:

In [8]:
crd.list_images(time="2019-04-28 00:00:00")

List of 1 images
 ↳[TessImage("tess2019117232932-s0011-2-2-0143-s_ffic.fits")]

You can access the image attributes using standard syntax:

In [9]:
img = crd.list_images(time="2019-04-28 00:00:00")[0]
img.sector, img.camera, img.ccd, img.time

(11, 2, 2, '2019-04-27 23:49:50.000')

You can also obtain the full URL of the image:

In [10]:
img.url

's3://stpubdata/tess/public/ffi/s0011/2019/117/2-2/tess2019117232932-s0011-2-2-0143-s_ffic.fits'

You can expert the image list as a Pandas DataFrame:

In [11]:
crd.list_images().to_pandas()

,url,sector,camera,ccd,time,cadenceno,quality
0,s3://stpubdata/tess/public/ffi/s0011/2019/113/...,11,2,2,2019-04-23 06:49:41.000,0,32
1,s3://stpubdata/tess/public/ffi/s0011/2019/113/...,11,2,2,2019-04-23 07:19:41.000,0,0
2,s3://stpubdata/tess/public/ffi/s0011/2019/113/...,11,2,2,2019-04-23 07:49:41.000,0,0
3,s3://stpubdata/tess/public/ffi/s0011/2019/113/...,11,2,2,2019-04-23 08:19:41.000,0,0
4,s3://stpubdata/tess/public/ffi/s0011/2019/113/...,11,2,2,2019-04-23 08:49:41.000,0,0
...,...,...,...,...,...,...,...
1243,s3://stpubdata/tess/public/ffi/s0011/2019/140/...,11,2,2,2019-05-20 06:50:08.000,0,0
1244,s3://stpubdata/tess/public/ffi/s0011/2019/140/...,11,2,2,2019-05-20 07:20:08.000,0,0
1245,s3://stpubdata/tess/public/ffi/s0011/2019/140/...,11,2,2,2019-05-20 07:50:08.000,0,0
1246,s3://stpubdata/tess/public/ffi/s0011/2019/140/...,11,2,2,2019-05-20 08:20:08.000,0,0


The full FFI file is often very big.  For this reason, the `TessImage` class also provides convenience methods to access exactly those parts of an image you need.

For example, the `read_header` method gives you fast access to a FITS header by reading only the first few kilobytes of the file from the cloud:

In [12]:
hdr = img.read_header(ext=0)
type(hdr)

astropy.io.fits.header.Header

You can download the WCS in a similar way:

In [13]:
wcs = img.read_wcs()
type(wcs)

astropy.wcs.wcs.WCS

You can download a specific part of the image using the `cutout` method:

In [14]:
img.cutout(column=200, row=300, shape=(3, 3)).flux

array([[603.30780029, 577.19464111, 533.54309082],
       [575.7154541 , 556.78961182, 520.02966309],
       [570.03234863, 512.32965088, 487.34893799]])

You do have the option to download the entire file using the `read` method, which returns an AstroPy FITS object:

In [15]:
img.read()

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x150ed8fd0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x150ed66d0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x150e7fd60>]

## Example 3: Creating your own `TessCoord` object

Note that you don't have to use the `locate` function to obtain a `TessCoord` object.  You can create your own as follows:

In [16]:
from tess_locator import TessCoord
TessCoord(sector=1, camera=1, ccd=1, column=50, row=70)

TessCoord(sector=1, camera=1, ccd=1, column=50.0, row=70.0, time=None)

## FAQ

### How does this package differ from `tess-point`?

`tess-locator` builds upon [`tess-point`](https://github.com/christopherburke/tess-point), i.e., `tess-point` is used behind the scenes for all transformations between pixel and celestial coordinates. Compared to `tess-point`, this package adds a user-friendly API, the ability to query positions by time (which is important for e.g. asteroids), and convenient access to FFI images.

### How does this package relate to other packages?

* [astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html) includes the excellent ``TesscutClass.get_sectors()`` method which queries a web API to inform you which sectors a position has been observed in. This package provides an offline version of that service, and adds the ability to query by time.
* [tess-waldo](https://github.com/SimonJMurphy/tess-waldo) lets you visualize how a target moves over the detector across sectors. It queries the ``TessCut`` service to obtain this information. This package adds the ability to create such plots offline.
